<a href="https://colab.research.google.com/github/ioiein/Advanced-ML/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import pandas as pd
import scipy
import sklearn
import sklearn.linear_model
import sklearn.preprocessing
import itertools

**Пункт 1**.\
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.


In [2]:
with open('/content/drive/MyDrive/chgk/results.pkl','rb') as f:
  results = pickle.load(f)
with open('/content/drive/MyDrive/chgk/players.pkl','rb') as f:
  players = pickle.load(f)
with open('/content/drive/MyDrive/chgk/tournaments.pkl','rb') as f:
  tournaments = pickle.load(f)

In [3]:
df_players = pd.DataFrame.from_dict(players, orient='index').set_index('id')
df_tournaments = pd.DataFrame.from_dict(tournaments, orient='index').set_index('id')

In [4]:
tournaments_2019 = df_tournaments[df_tournaments['dateStart'].str[0:4] == "2019"]
tournaments_2020 = df_tournaments[df_tournaments['dateStart'].str[0:4] == "2020"]

In [5]:
tournaments_2019.head(5)

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
4772,Синхрон северных стран. Зимний выпуск,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,Балтийский Берег. 3 игра,2019-01-25T19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,Балтийский Берег. 4 игра,2019-03-01T19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,Балтийский Берег. 5 игра,2019-04-05T19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,ОВСЧ. 6 этап,2019-02-15T20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [6]:
tournaments_2020.head(5)

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
4628,Семь сорок,2020-12-30T16:00:00+03:00,2020-12-30T16:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",None,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",{'dateRequestsAllowedTo': '2020-12-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
4957,Синхрон Биркиркары,2020-02-21T00:00:00+03:00,2020-02-27T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5151,Яровой,2020-08-01T14:00:00+03:00,2020-08-05T14:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 22325, 'name': 'Михаил', 'patronymic':...",{'dateRequestsAllowedTo': '2020-07-24T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5414,Синхрон северных стран,2020-01-03T19:00:00+03:00,2020-01-10T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5477,Онлайн: Синхрон Урюбджирова,2020-04-18T19:00:00+03:00,2020-04-30T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 91324, 'name': 'Эрдни', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-04-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"


In [7]:
#составим DataFrame с полями tournament_id, team_id, position, player_id, mask и mask_len(она понадобится для выкидывания турниров с неодинаковыми масками)
tournament_list = []
player_list = []
team_list = []
mask_list = []
pos_list = []

for tournament_id in results.keys():
    if (tournament_id in np.concatenate((tournaments_2019.index, tournaments_2020.index))):
        for team in results[tournament_id]:
            if 'mask' in team:
                mask = team['mask']
                position = team['position']
                team_id = team['team']['id']
                for member in team['teamMembers']:
                    tournament_list.append(tournament_id)
                    mask_list.append(mask)
                    pos_list.append(position)
                    team_list.append(team_id)
                    player_list.append(member['player']['id'])
df_players_in_tournaments = pd.DataFrame({'tournament_id': tournament_list,
                   'team_id': team_list,
                   'position': pos_list,
                   'player_id': player_list,
                   'mask': mask_list
             })
df_players_in_tournaments = df_players_in_tournaments[~df_players_in_tournaments['mask'].isna()]
df_players_in_tournaments['mask_len'] = df_players_in_tournaments['mask'].str.len()

#оставляем только турниры с одинаковыми масками
same_masks = df_players_in_tournaments.groupby('tournament_id')['mask_len'].nunique().eq(1)
same_masks = same_masks[same_masks]
df_players_in_tournaments = df_players_in_tournaments[df_players_in_tournaments['tournament_id'].isin(same_masks.index)]

In [8]:
df_players_in_tournaments.head(5)

,tournament_id,team_id,position,player_id,mask,mask_len
0,4772,45556,1.0,6212,111111111011111110111111111100010010,36
1,4772,45556,1.0,18332,111111111011111110111111111100010010,36
2,4772,45556,1.0,18036,111111111011111110111111111100010010,36
3,4772,45556,1.0,22799,111111111011111110111111111100010010,36
4,4772,45556,1.0,15456,111111111011111110111111111100010010,36


In [9]:
questions = []
for l in df_players_in_tournaments['mask_len']:
    questions.extend(np.arange(1, l + 1))
    
player_question = pd.DataFrame({
    'tournament_id': np.repeat(df_players_in_tournaments['tournament_id'], df_players_in_tournaments['mask_len']),
    'team_id': np.repeat(df_players_in_tournaments['team_id'], df_players_in_tournaments['mask_len']),
    'position': np.repeat(df_players_in_tournaments['position'], df_players_in_tournaments['mask_len']),
    'player_id': np.repeat(df_players_in_tournaments['player_id'], df_players_in_tournaments['mask_len']),
    'question_number': questions,
    'correct': list(itertools.chain.from_iterable(df_players_in_tournaments['mask']))
})
player_question['tournament_question'] = player_question['tournament_id'].astype(str) + '_' + player_question['question_number'].astype(str)

bad_ans = player_question[player_question['correct'].isin(['?', 'X'])] # в mask бывают символы Х и ? для некоторых вопросов, их выкидываю
player_question = player_question[~player_question['tournament_question'].isin(bad_ans['tournament_question'])]
player_question.loc[:, 'correct'] = player_question['correct'].astype(int, copy=False)

del df_players_in_tournaments

In [10]:
player_question.head(5)

,tournament_id,team_id,position,player_id,question_number,correct,tournament_question
0,4772,45556,1.0,6212,1,1,4772_1
0,4772,45556,1.0,6212,2,1,4772_2
0,4772,45556,1.0,6212,3,1,4772_3
0,4772,45556,1.0,6212,4,1,4772_4
0,4772,45556,1.0,6212,5,1,4772_5


**Пункт 2.**\
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их 
предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.





In [11]:
train = player_question[player_question['tournament_id'].isin(tournaments_2019.index.values)]
test = player_question[player_question['tournament_id'].isin(tournaments_2020.index.values)]

del player_question

In [12]:
#при помощи onehotencoder приводим пары игрок-вопрос к sparse матрице и обучаем при помощи LogosticRegression из sklearn
ohe = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train[['player_id', 'tournament_question']])
y_train = train['correct']

In [13]:
baseline_model = sklearn.linear_model.LogisticRegression(max_iter=300, n_jobs=-1)
baseline_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
baseline_model_weights = dict(zip(ohe.categories_[0], baseline_model.coef_[0][:ohe.categories_[0].shape[0]]))

In [15]:
player_rating = train[['player_id']].drop_duplicates().reset_index(drop=True)

In [16]:
player_rating['baseline_rating'] = player_rating['player_id'].map(baseline_model_weights)
player_rating = player_rating.merge(df_players, left_on='player_id', right_index=True)

In [17]:
player_rating.sort_values(by='baseline_rating').tail(20)[['surname', 'name', 'patronymic']][::-1] 
# топ-20 игроков, тут присутствует достаточно игроков из реального топа

,surname,name,patronymic
8222,Руссо,Максим,Михайлович
5957,Брутер,Александра,Владимировна
1209,Семушин,Иван,Николаевич
8244,Спешков,Сергей,Леонидович
1208,Савченков,Михаил,Владимирович
155,Сорожкин,Артём,Сергеевич
1210,Мереминский,Станислав,Григорьевич
2,Левандовский,Михаил,Ильич
3,Николенко,Сергей,Игоревич
5,Прокофьева,Ирина,Сергеевна


**Пункт 3.**\
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Из предказаний baseline модели мы знаем вероятность каждого игрока ответить на вопрос. Обозначим вероятность  правильного ответа i-го игрока за $p_i$. Тогда вероятность команды ответить на вопрос $P_{team} = 1 - \prod\limits_{i \in team}(1 - p_i)$

In [18]:
def spearman_and_kendall_correlations(test_prediction_prob):
    df_prediction_ranks = test[['tournament_id', 'team_id']]
    df_prediction_ranks['probability_incorrect'] = 1 - test_prediction_prob
    df_prediction_ranks = df_prediction_ranks.groupby(['tournament_id', 'team_id']).prod().reset_index()
    df_prediction_ranks['position_prediction'] = df_prediction_ranks.groupby('tournament_id')['probability_incorrect'].rank('dense')
    
    df_true_ranks = test[['tournament_id', 'team_id', 'position']].drop_duplicates()
    df_ranks = pd.merge(df_prediction_ranks, df_true_ranks, on=['tournament_id', 'team_id'])

    spearman_correlations = []
    kendall_correlations = []
    for tournament_id in df_ranks['tournament_id'].unique():
        current_tournament = df_ranks[df_ranks['tournament_id'] == tournament_id]
        if len(current_tournament) > 1: #беру турниры с более чем одной командой иначе nan получается
          spearman_correlations.append(scipy.stats.spearmanr(current_tournament['position'], current_tournament['position_prediction'])[0])
          kendall_correlations.append(scipy.stats.kendalltau(current_tournament['position'], current_tournament['position_prediction'])[0])
    return np.mean(spearman_correlations), np.mean(kendall_correlations)

In [19]:
test_prediction_probability = baseline_model.predict_proba(ohe.transform(test[['player_id', 'tournament_question']]))[:, 1]
spearman_correlation, kendall_correlation = spearman_and_kendall_correlations(test_prediction_probability)
print("Корреляции результатов турниров в 2020:")
print("Спирмена: ", spearman_correlation, "Кендалла: ", kendall_correlation)
# корреляции получились в пределах нормы, указанных в условии

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Корреляции результатов турниров в 2020:
Спирмена:  0.7761154231704326 Кендалла:  0.6180683318356113


**Пункт 4.**\
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; 
* не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


In [20]:
def E_step(train_):
  train_['probability_incorrect'] = 1 - train_['player_prediction']
  teams = 1 - train_.groupby(['tournament_id', 'team_id', 'question_number'])['probability_incorrect'].prod()
  train_ = train_.merge(teams.rename('team_prediction'), left_on=['tournament_id', 'team_id', 'question_number'], right_index=True)
  train_['missing_values'] = train_['player_prediction'] / train_['team_prediction']
  train_['missing_values'] = np.where(y_train == 0, 0, train_['missing_values'])
  train_['missing_values'] = np.clip(train_['missing_values'], 1e-8, 1 - 1e-8)
  return train_

In [21]:
def M_step(model, train_):
  model.fit(X_train, train_['missing_values'])
  train_['player_prediction'] = model.predict_proba(X_train)[:, 1]
  train_ = train_.drop('team_prediction',1)
  return model, train_

In [ ]:
train['player_prediction'] = baseline_model.predict_proba(X_train)[:, 1]
EM_model = sklearn.linear_model.LinearRegression(n_jobs=-1)
for i in range(5):
  train['probability_incorrect'] = 1 - train['player_prediction']
  teams = 1 - train.groupby(['tournament_id', 'team_id', 'question_number'])['probability_incorrect'].prod()
  train = train.merge(teams.rename('team_prediction'), left_on=['tournament_id', 'team_id', 'question_number'], right_index=True)
  train['missing_values'] = train['player_prediction'] / train['team_prediction']
  train['missing_values'] = np.where(y_train == 0, 0, train['missing_values'])
  train['missing_values'] = np.clip(train['missing_values'], 1e-8, 1 - 1e-8)

  EM_model.fit(X_train, train['missing_values'])
  train['player_prediction'] = EM_model.predict(X_train)
  train = train.drop('team_prediction',1)
  
  print('Шаг', i)
  test_prediction_probability = EM_model.predict(X_test)
  spearman_correlation, kendall_correlation = get_correlations(test_prediction_probability)

  print('Корреляции:')
  print('Спирмена: ', spearman_correlation, 'Кендалла: ', kendall_correlation)